In [1]:
def read_file(_path, delim='\t'):
    data = []
    if delim == 'tab':
        delim = '\t'
    elif delim == 'space':
        delim = ' '
    with open(_path, 'r') as f:
        for line in f:
            line = line.strip().split(delim)
            line = [float(i) for i in line]
            data.append(line)
    return np.asarray(data)

In [2]:
def poly_fit(traj, traj_len, 
             threshold
             ):
    """
    Input:
    - traj: Numpy array of shape (2, traj_len)
    - traj_len: Len of trajectory
    - threshold: Minimum error to be considered for non linear traj
    Output:
    - int: 1 -> Non Linear 0-> Linear
    """
    t = np.linspace(0, traj_len - 1, traj_len) # np.linspace(시작점, 끝점, 구간 내 숫자 개수)
    res_x = np.polyfit(t, traj[0, -traj_len:], 2, full=True)[1]
    res_y = np.polyfit(t, traj[1, -traj_len:], 2, full=True)[1]
    if res_x + res_y >= threshold: # error
       return 1.0
    else:
       return 0.0
    return 0.0

In [3]:
import logging
import os
import math

import numpy as np


import torch
from torch.utils.data import Dataset
path = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep.txt'
path2 = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep2.txt'
path3 = 'C:\\Users\\NGN\\dev\\Traffino\\TRAFFINO\\traffino\\datasets\\waterloo\\train\\0769_prep3.txt'
delim = '\t'
data = read_file(path, delim)
data2 = read_file(path2, delim)
data3 = read_file(path3, delim)
print(len(data))

print(len(data))
print(type(data))

16387
16387
<class 'numpy.ndarray'>


In [4]:
frames = np.unique(data[:, 0]).tolist()
print(frames[0])
print(len(frames))

1.0
727


In [5]:
frame_data_all = []
frame_data = []
state_data = []
traffic_data = []
# data를 frame 단위로 변환
for frame in frames:
    frame_data_all.append(data[frame == data[:, 0], :4])  # all
    frame_data.append(data[frame == data[:, 0], :4]) # 전체 frame에(7994개 scene) 대해, 각 frame별 frame_data (agent 정보, x, y) --> 2차원 list
    state_data.append(data2[frame == data2[:, 0], :])
    traffic_data.append(data3[frame == data3[:, 0], ])

In [6]:
print(len(frame_data))
print(type(frame_data))
print(frame_data)
# print(len(frame_data[0]))

727
<class 'list'>
[array([[1.00000000e+00, 1.00000000e+00, 5.38830520e+05, 4.81401216e+06],
       [1.00000000e+00, 2.00000000e+00, 5.38801510e+05, 4.81398483e+06],
       [1.00000000e+00, 3.00000000e+00, 5.38839930e+05, 4.81399724e+06],
       [1.00000000e+00, 4.00000000e+00, 5.38853070e+05, 4.81402927e+06],
       [1.00000000e+00, 5.00000000e+00, 5.38807070e+05, 4.81399478e+06],
       [1.00000000e+00, 6.00000000e+00, 5.38851070e+05, 4.81402437e+06],
       [1.00000000e+00, 7.00000000e+00, 5.38865730e+05, 4.81403121e+06],
       [1.00000000e+00, 8.00000000e+00, 5.38826550e+05, 4.81399631e+06],
       [1.00000000e+00, 9.00000000e+00, 5.38858290e+05, 4.81402357e+06],
       [1.00000000e+00, 1.00000000e+01, 5.38819410e+05, 4.81399724e+06],
       [1.00000000e+00, 1.10000000e+01, 5.38842390e+05, 4.81400065e+06],
       [1.00000000e+00, 1.20000000e+01, 5.38845550e+05, 4.81402547e+06],
       [1.00000000e+00, 1.30000000e+01, 5.38859360e+05, 4.81402795e+06],
       [1.00000000e+00, 1.40000

In [7]:
print(len(state_data))
print(type(state_data))
print(state_data)
# print(len(state_data[0]))

727
<class 'list'>
[array([[ 1.00000e+00,  1.00000e+00,  6.11800e+01, -2.24500e-01,
         1.16600e-01,  5.29890e+00],
       [ 1.00000e+00,  2.00000e+00,  2.14122e+01,  4.90000e-02,
         4.00000e-03,  4.34800e-01],
       [ 1.00000e+00,  3.00000e+00,  5.31595e+01, -1.74100e-01,
         2.60000e-03,  5.30940e+00],
       [ 1.00000e+00,  4.00000e+00,  1.50000e-03,  1.50000e-03,
        -1.30000e-03,  3.59490e+00],
       [ 1.00000e+00,  5.00000e+00,  2.63638e+01,  1.08700e-01,
        -8.30000e-03,  4.46300e-01],
       [ 1.00000e+00,  6.00000e+00,  3.90000e-03,  2.80000e-03,
         5.80000e-03,  3.66580e+00],
       [ 1.00000e+00,  7.00000e+00,  2.00000e-03, -4.10000e-03,
        -3.60000e-03,  3.60680e+00],
       [ 1.00000e+00,  8.00000e+00,  3.95930e+00,  8.30000e-03,
        -6.49000e-02,  1.48700e-01],
       [ 1.00000e+00,  9.00000e+00,  2.10000e-03,  2.10000e-03,
        -2.10000e-03,  3.57830e+00],
       [ 1.00000e+00,  1.00000e+01,  5.32860e+00,  3.15000e-02,
       

In [8]:
print(len(traffic_data))
print(type(traffic_data))
print(traffic_data)
# print(len(state_data[0]))

727
<class 'list'>
[array([[ 1.,  1.,  0.],
       [ 1.,  2.,  0.],
       [ 1.,  3.,  0.],
       [ 1.,  4.,  0.],
       [ 1.,  5.,  0.],
       [ 1.,  6.,  0.],
       [ 1.,  7.,  0.],
       [ 1.,  8.,  0.],
       [ 1.,  9.,  0.],
       [ 1., 10.,  0.],
       [ 1., 11.,  0.],
       [ 1., 12.,  0.],
       [ 1., 13.,  0.],
       [ 1., 14.,  0.],
       [ 1., 15.,  0.],
       [ 1., 16.,  0.],
       [ 1., 17.,  0.],
       [ 1., 18.,  0.],
       [ 1., 19.,  0.]]), array([[ 2.,  1.,  0.],
       [ 2.,  2.,  0.],
       [ 2.,  3.,  0.],
       [ 2.,  4.,  0.],
       [ 2.,  5.,  0.],
       [ 2.,  6.,  0.],
       [ 2.,  7.,  0.],
       [ 2.,  8.,  0.],
       [ 2.,  9.,  0.],
       [ 2., 10.,  0.],
       [ 2., 11.,  0.],
       [ 2., 12.,  0.],
       [ 2., 13.,  0.],
       [ 2., 14.,  0.],
       [ 2., 15.,  0.],
       [ 2., 16.,  0.],
       [ 2., 17.,  0.],
       [ 2., 18.,  0.],
       [ 2., 19.,  0.],
       [ 2., 20.,  0.]]), array([[ 3.,  1.,  0.],
       [ 3.,  2.

In [13]:
# print(frame_data[0:20])
# print(len(frame_data[0:20]))

In [9]:
seq_len=20
skip=1

num_sequences = int(math.ceil((len(frames) - seq_len + 1) / skip)) # seqence 개수

In [10]:
obs_len = 8
pred_len = 12
threshold = 0.002
min_agent =1 
num_agents_in_seq = []
seq_list = []
seq_list2 = [] # state
seq_list3 = [] # traffic 

non_linear_agent = []
loss_mask_list = []
seq_list_rel = []
seq_list_rel2 = [] # state
seq_list_rel3 = [] # traffic 



for idx in range(0, num_sequences * skip + 1, skip):
    curr_seq_data = np.concatenate(                         # num_sequnce만큼 끊어서 curr seq_data 생성
        frame_data[idx:idx + seq_len], axis=0               # ex) seq 1: 1-20 frame, seq 2: 2-21 frame ... 
    )
    curr_seq_data2 = np.concatenate(                         # num_sequnce만큼 끊어서 curr seq_data 생성
        state_data[idx:idx + seq_len], axis=0               # ex) seq 1: 1-20 frame, seq 2: 2-21 frame ... 
    )
    curr_seq_data3 = np.concatenate(                         # num_sequnce만큼 끊어서 curr seq_data 생성
        traffic_data[idx:idx + seq_len], axis=0               # ex) seq 1: 1-20 frame, seq 2: 2-21 frame ... 
    )
    # print(idx) # 7976 (sequnece 개수)
    # print(curr_seq_data)
    # print(len(curr_seq_data)) # 현재 시퀀스에 있는 row (프레임 별 agent)의 합 --> 20개 프레임에 대한 agent (중복 포함) 합 --> 20*20= 400

    agents_in_curr_seq = np.unique(curr_seq_data[:, 1])       # 현재 seq에 있는 agents 목록, 모든 행의 1번째(agent 정보) 열 slicing

    # print(len(agents_in_curr_seq))
    # print(agents_in_curr_seq)
    curr_seq_rel = np.zeros((len(agents_in_curr_seq), 2,        # (현재 seq에 있는 agents 개수, 2, seq_len) 
                                seq_len))
    curr_seq_rel2 = np.zeros((len(agents_in_curr_seq), 4,        # (현재 seq에 있는 agents 개수, 4, seq_len) --> state 저장용
                            seq_len))
    curr_seq_rel3 = np.zeros((len(agents_in_curr_seq), 1,        # (현재 seq에 있는 agents 개수, 1, seq_len) --> traffic light 저장용
                        seq_len))
    
    # print(curr_seq_rel)
    curr_seq = np.zeros((len(agents_in_curr_seq), 2, seq_len))   # (현재 seq에 있는 agents 개수, 2, seq_len) 
    curr_seq2 = np.zeros((len(agents_in_curr_seq), 4, seq_len))   # (현재 seq에 있는 agents 개수, 4, seq_len) --> state 저장용
    curr_seq3 = np.zeros((len(agents_in_curr_seq), 1, seq_len))   # (현재 seq에 있는 agents 개수, 3, seq_len) --> traffic light 저장용
    
    curr_loss_mask = np.zeros((len(agents_in_curr_seq),
                                seq_len))
    num_agents_considered = 0
    _non_linear_agent = []
    
    for _, agent_id in enumerate(agents_in_curr_seq):           # 현재 seq에 있는 agents 목록
        curr_agent_seq = curr_seq_data[ curr_seq_data[:, 1] == # 현재 seq에 있는 agent의 위치 seqeunce curr_agent_seq: (20, 4)
                                        agent_id, :]
        curr_agent_seq2 = curr_seq_data2[ curr_seq_data2[:, 1] == # 현재 seq에 있는 agent의 상태 seqeunce curr_agent_seq: (20, 4)
                                agent_id, :]
        curr_agent_seq3 = curr_seq_data3[ curr_seq_data3[:, 1] == # 현재 seq에 있는 agent의 traffic seqeunce curr_agent_seq: (20, 4)
                                agent_id, :]
        # print(curr_agent_seq)
        # print(agent_id)
        curr_agent_seq = np.around(curr_agent_seq, decimals=4)
        curr_agent_seq2 = np.around(curr_agent_seq2, decimals=4)
        curr_agent_seq3 = np.around(curr_agent_seq3, decimals=4)
                
        pad_front = frames.index(curr_agent_seq[0, 0]) - idx
        pad_end = frames.index(curr_agent_seq[-1, 0]) - idx + 1
        if pad_end - pad_front != seq_len:
            continue
        curr_agent_seq = np.transpose(curr_agent_seq[:, 2:])
        curr_agent_seq2 = np.transpose(curr_agent_seq2[:, 2:])
        curr_agent_seq3 = np.transpose(curr_agent_seq3[:, 2:])
        
        curr_agent_seq = curr_agent_seq
        # Make coordinates relative
        rel_curr_agent_seq = np.zeros(curr_agent_seq.shape)
        
        rel_curr_agent_seq[:, 1:] = \
            curr_agent_seq[:, 1:] - curr_agent_seq[:, :-1]
        _idx = num_agents_considered
        
        curr_seq[_idx, :, pad_front:pad_end] = curr_agent_seq
        curr_seq2[_idx, :, pad_front:pad_end] = curr_agent_seq2
        curr_seq3[_idx, :, pad_front:pad_end] = curr_agent_seq3
        
        curr_seq_rel[_idx, :, pad_front:pad_end] = rel_curr_agent_seq
        # Linear vs Non-Linear Trajectory
        _non_linear_agent.append(
            poly_fit(curr_agent_seq, pred_len 
                        ,threshold
                        )
                        )
        curr_loss_mask[_idx, pad_front:pad_end] = 1
        num_agents_considered += 1
        
    if num_agents_considered > min_agent:
        non_linear_agent += _non_linear_agent
        num_agents_in_seq.append(num_agents_considered)
        loss_mask_list.append(curr_loss_mask[:num_agents_considered])
        seq_list.append(curr_seq[:num_agents_considered])
        seq_list2.append(curr_seq2[:num_agents_considered])
        seq_list3.append(curr_seq3[:num_agents_considered])
        
        seq_list_rel.append(curr_seq_rel[:num_agents_considered])
        seq_list_rel2.append(curr_seq_rel2[:num_agents_considered])
        seq_list_rel3.append(curr_seq_rel3[:num_agents_considered])

num_seq = len(seq_list)
        
seq_list = np.concatenate(seq_list, axis=0)
seq_list2 = np.concatenate(seq_list2, axis=0)
seq_list3 = np.concatenate(seq_list3, axis=0)
seq_list_rel = np.concatenate(seq_list_rel, axis=0)
seq_list_rel2 = np.concatenate(seq_list_rel2, axis=0)
seq_list_rel3 = np.concatenate(seq_list_rel3, axis=0)

loss_mask_list = np.concatenate(loss_mask_list, axis=0)
non_linear_agent = np.asarray(non_linear_agent)

# Convert numpy -> Torch Tensor
obs_traj = torch.from_numpy(
    seq_list[:, :, :obs_len]).type(torch.float)
obs_state = torch.from_numpy(
    seq_list2[:, :, :obs_len]).type(torch.float)
# obs_traffic = torch.from_numpy(
#     seq_list3[:, :, :obs_len]).type(torch.float)

pred_traj = torch.from_numpy(
    seq_list[:, :, obs_len:]).type(torch.float)
pred_state = torch.from_numpy(
    seq_list2[:, :, obs_len:]).type(torch.float)
# pred_traffic = torch.from_numpy(
#     seq_list3[:, :, obs_len:]).type(torch.float)

obs_traj_rel = torch.from_numpy(
    seq_list_rel[:, :, :obs_len]).type(torch.float)
obs_state_rel = torch.from_numpy(
    seq_list_rel2[:, :, :obs_len]).type(torch.float)
# obs_traffic_rel = torch.from_numpy(
#     seq_list_rel3[:, :, :obs_len]).type(torch.float)

pred_traj_rel = torch.from_numpy(
    seq_list_rel[:, :, obs_len:]).type(torch.float)
pred_state_rel = torch.from_numpy(
    seq_list_rel2[:, :, obs_len:]).type(torch.float)
# pred_traffic_rel = torch.from_numpy(
#     seq_list_rel3[:, :, obs_len:]).type(torch.float)

loss_mask = torch.from_numpy(loss_mask_list).type(torch.float)
non_linear_agent = torch.from_numpy(non_linear_agent).type(torch.float)
cum_start_idx = [0] + np.cumsum(num_agents_in_seq).tolist()
seq_start_end = [
    (start, end)
    for start, end in zip(cum_start_idx, cum_start_idx[1:])
]

def len():              # len(train_dset)
    return num_seq

def getitem(index):
    start, end = seq_start_end[index]
    out = [
        obs_traj[start:end, :], 
        obs_state[start:end, :],
        pred_traj[start:end, :],
        pred_state[start:end, :],
        obs_traj_rel[start:end, :], 
        obs_state_rel[start:end, :], 
        pred_traj_rel[start:end, :],
        pred_state_rel[start:end, :],
        non_linear_agent[start:end], 
        loss_mask[start:end, :]
    ]
    return out